In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
#!unzip household_power_consumption.zip
#!pip3 install prophet
#!pip install prophet

In [29]:
pd.set_option("display.float_format", "{:.3f}".format)

In [30]:
# The ds (datestamp) column should be of a format expected by Pandas,
# ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

df = pd.read_csv('household_power_consumption.txt', sep=';')

C:\Users\jaros\AppData\Local\Temp\ipykernel_10668\293355836.py:4: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('household_power_consumption.txt', sep=';')


In [31]:
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])

del df['Date']
del df['Time']

df.set_index("DateTime", inplace=True)

C:\Users\jaros\AppData\Local\Temp\ipykernel_10668\1650630806.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


In [32]:
df.isna().sum()

Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64

In [33]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Global_active_power,2075259,6534,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_reactive_power,2075259,896,0.000,472786,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Voltage,2075259,5168,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_intensity,2075259,377,1.000,169406,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_1,2075259,153,0.000,1840611,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_2,2075259,145,0.000,1408274,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_3,2049280.000,NaN,NaN,NaN,6.458,8.437,0.000,0.000,1.000,17.000,31.000


In [34]:
df.isin(['?']).sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3               0
dtype: int64

In [35]:
df.replace('?', 'NaN', inplace=True)

In [36]:
df['Global_active_power'] = df['Global_active_power'].astype('float')
df['Global_reactive_power'] = df['Global_reactive_power'].astype('float')
df['Voltage'] = df['Voltage'].astype('float')
df['Global_intensity'] = df['Global_intensity'].astype('float')
df['Sub_metering_1'] = df['Sub_metering_1'].astype('float')
df['Sub_metering_2'] = df['Sub_metering_2'].astype('float')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [38]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2049280.000,1.092,1.057,0.076,0.308,0.602,1.528,11.122
Global_reactive_power,2049280.000,0.124,0.113,0.000,0.048,0.100,0.194,1.390
Voltage,2049280.000,240.840,3.240,223.200,238.990,241.010,242.890,254.150
Global_intensity,2049280.000,4.628,4.444,0.200,1.400,2.600,6.400,48.400
Sub_metering_1,2049280.000,1.122,6.153,0.000,0.000,0.000,0.000,88.000
Sub_metering_2,2049280.000,1.299,5.822,0.000,0.000,0.000,1.000,80.000
Sub_metering_3,2049280.000,6.458,8.437,0.000,0.000,1.000,17.000,31.000


In [39]:
# Fill missing values with a value at the same time, one day ago

In [40]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [41]:
df = df.fillna(df.shift(60*24*7))

In [42]:
df.isna().sum()

Global_active_power      3
Global_reactive_power    3
Voltage                  3
Global_intensity         3
Sub_metering_1           3
Sub_metering_2           3
Sub_metering_3           3
dtype: int64

In [43]:
df = df.fillna(df.shift(60*24*7))

In [44]:
df.isna().sum()

Global_active_power      2
Global_reactive_power    2
Voltage                  2
Global_intensity         2
Sub_metering_1           2
Sub_metering_2           2
Sub_metering_3           2
dtype: int64

In [45]:
df = df.fillna(df.shift(60*24))

In [46]:
df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

In [47]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2075259.000,1.089,1.056,0.076,0.308,0.598,1.526,11.122
Global_reactive_power,2075259.000,0.124,0.113,0.000,0.048,0.100,0.194,1.390
Voltage,2075259.000,240.846,3.235,223.200,239.000,241.020,242.880,254.150
Global_intensity,2075259.000,4.617,4.439,0.200,1.400,2.600,6.400,48.400
Sub_metering_1,2075259.000,1.121,6.152,0.000,0.000,0.000,0.000,88.000
Sub_metering_2,2075259.000,1.296,5.814,0.000,0.000,0.000,1.000,80.000
Sub_metering_3,2075259.000,6.454,8.434,0.000,0.000,1.000,17.000,31.000


In [48]:
# Standardization (watt-hour)
df["Global_active_power"] = df["Global_active_power"].apply(lambda x: (x*1000)/60)
df["Global_reactive_power"] = df["Global_reactive_power"].apply(lambda x: (x*1000)/60)

In [49]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2075259.000,18.151,17.601,1.267,5.133,9.967,25.433,185.367
Global_reactive_power,2075259.000,2.063,1.879,0.000,0.800,1.667,3.233,23.167
Voltage,2075259.000,240.846,3.235,223.200,239.000,241.020,242.880,254.150
Global_intensity,2075259.000,4.617,4.439,0.200,1.400,2.600,6.400,48.400
Sub_metering_1,2075259.000,1.121,6.152,0.000,0.000,0.000,0.000,88.000
Sub_metering_2,2075259.000,1.296,5.814,0.000,0.000,0.000,1.000,80.000
Sub_metering_3,2075259.000,6.454,8.434,0.000,0.000,1.000,17.000,31.000


In [50]:
# Global_active_power   [watt-hour]
# Global_reactive_power [watt-hour]
# Voltage               [volt]
# Global_intensity      [ampere]
# Sub_metering_1        [watt-hour]
# Sub_metering_2        [watt-hour]
# Sub_metering_3        [watt-hour]

In [51]:

df = df[['Global_active_power','Sub_metering_1','Sub_metering_2','Sub_metering_3']]